# Tutorial 1 (Model)

This is a tutorial for E-Cell4. Here, we represent the way of modeling with E-Cell 4.

## Species

We'll show you how to create **Species**.

In [4]:
from ecell4 import *

A = Species("A")
B = Species("B")

each expression describes a **Species** named **A** or **B**.
**A** describes not a specific molecule, but a type of molecule.
#### Caution
Species name(we call this **Serial**) has a number of naming rules,
The naming requires attention to use special symbol '()', '.', '_', numbers, and space.

<hr>

You can add attributes to Species,

In [5]:
A = Species("A")
A.set_attribute("radius", "0.005")
A.set_attribute("D", "1")
A.set_attribute("location", "cytoplasm")

The 1st argument for **set_attribute** is the name of attribute.
The 2nd argument is the value.
Both have got to be string.

Radius, diffusion coefficient, location are frequently used, so there is a shortcut for this.

In [6]:
A = Species("A", "0.005", "1", "cytoplasm")  # XXX: serial, radius, D, location

When you want to inspect the **Species** attributes, please write as follows.

In [8]:
X = Species("A", "0.005", "1")

print(X.serial())  # will return 'A'
print(X.get_attribute("D"))  # will return '1'


A
1


In E-Cell4, NetworkModel consists of a set of ReactionRules (and Species attributes). First, instantiate NetworkModel.

In [1]:
from ecell4.core import *
m = NetworkModel()

Here, you can add five kinds of ReactionRules.

1. create_binding_reaction_rule
2. create_degradation_reaction_rule
3. create_synthesis_reaction_rule
4. create_unbinding_reaction_rule
5. create_unimolecular_reaction_rule

Now, for the simple binding/unbinding equilibrium model, let's use 1 and 4.

In [2]:
rr1 = create_binding_reaction_rule(Species("A"), Species("B"), Species("C"), 1.0)
rr2 = create_unbinding_reaction_rule(Species("C"), Species("A"), Species("B"), 1.0)

A ReactionRule consists of reactants, products, and a kinetic rate. Of course, you can change the kinetic rate.

In [3]:
print len(rr1.reactants()), len(rr1.products()), rr1.k()
print len(rr2.reactants()), len(rr2.products()), rr2.k()
rr2.set_k(2.0)
print rr2.k()

2 1 1.0
1 2 1.0
2.0


You can get a simple string representing a ReactionRule by using as_string method.

In [4]:
print rr1.as_string()
print rr2.as_string()

A+B>C|1
C>A+B|2


Next, add these ReactionRules into a NetworkModel.

In [5]:
m.add_reaction_rule(rr1)
m.add_reaction_rule(rr2)
print len(m.reaction_rules())
print m.has_reaction_rule(rr1), m.has_reaction_rule(rr2)

2
True True


NetworkModel returns appropriate ReactionRules from the given Species through APIs named query_reaction_rules.

In [6]:
print [rr.as_string() for rr in m.query_reaction_rules(Species("C"))]
print [rr.as_string() for rr in m.query_reaction_rules(Species("A"))]
print [rr.as_string() for rr in m.query_reaction_rules(Species("A"), Species("B"))]
print [rr.as_string() for rr in m.query_reaction_rules(Species("B"), Species("A"))]

['C>A+B|2']
[]
['A+B>C|1']
['A+B>C|1']


NetworkModel also contains Species attributes. These attributes are indispensable for particle and lattice simulations, but not necessarily needed for gillespie and ode.

In [7]:
sp1 = Species("A")
sp1.set_attribute("radius", "0.0025")
sp1.set_attribute("D", "1")
m.add_species_attribute(sp1)
# m.add_species_attribute(Species("A", "0.0025", "1"))

NetworkModel attributes a Species based on the registered Species.

In [8]:
print m.has_species_attribute(Species("A"))
sp2 = m.apply_species_attributes(Species("A"))
print sp2.has_attribute("radius"), sp2.has_attribute("D")
print sp2.get_attribute("radius"), sp2.get_attribute("D")

True
True True
0.0025 1
